In [1]:
%cd ~/corpus

/home/sibava/corpus


In [2]:
import json
import jsonlines
import pandas as pd
from transformers import T5Tokenizer
from pprint import pprint
import copy
import itertools
from tqdm.auto import tqdm

In [3]:
tokenizer = T5Tokenizer.from_pretrained('megagonlabs/t5-base-japanese-web')

In [4]:
case_en_ja = {
	'ga':'が',
	'o':'を',
	'ni':'に'
}

In [5]:
psa_df = pd.read_json("/home/sibava/corpus/psa_instanceC.jsonl",orient='records',lines=True)
outputfile = open("/home/sibava/corpus/psat5instance.jsonl",mode='w')

In [6]:
psa_df['arg_surface'].mask(psa_df['arg_type'] == 'exog','<extra_id_99>',inplace=True)
psa_df['arg_surface'].mask(psa_df['arg_type'] == 'exo1','<extra_id_98>',inplace=True)
psa_df['arg_surface'].mask(psa_df['arg_type'] == 'exo2','<extra_id_97>',inplace=True)

In [23]:
ga_df = psa_df.iloc[::3,].reset_index(drop=True)
o_df = psa_df.iloc[1::3,].reset_index(drop=True)
ni_df = psa_df.iloc[2::3,].reset_index(drop=True)

In [24]:
for df,case in zip([ga_df,o_df,ni_df],['が','を','に']):
	df['arg_surface'].mask(df['arg_type'] == 'none','',inplace=True)
	df['arg_surface'].where(df['arg_type'] == 'none',df['arg_surface'] + case,inplace=True)

In [26]:
inputs = []
for context,pred_sent_index,pred_indices in zip(ga_df['context'].values,ga_df['pred_sent_index'].values,ga_df['pred_indices'].values):
	print(context,pred_indices,pred_sent_index)
	context[pred_sent_index].insert(pred_indices[0],"<extra_id_0>")
	context[pred_sent_index].insert(pred_indices[0],"<extra_id_0>")
	inputs.append(context)
	break

[['村山', '富市', '首相', 'は', '年頭', 'に', 'あたり', '首相', '官邸', 'で', '内閣', '記者', '会', 'と', '二十八', '日', '<extra_id_0>', '<extra_id_0>', '<extra_id_0>', '<extra_id_0>', '<extra_id_0>', '<extra_id_0>', '<extra_id_0>', '<extra_id_0>', '<extra_id_0>', '<extra_id_0>', '<extra_id_0>', '<extra_id_0>', '会見', 'し', '、', '社会党', 'の', '新', '民主', '連合', '所属', '議員', 'の', '離党', '問題', 'に', 'ついて', '「', '政権', 'に', '影響', 'を', '及ぼす', 'こと', 'に', 'は', 'なら', 'ない', '。', '離党', '者', 'が', 'いて', 'も', '、', 'その', '範囲', 'に', 'とどまる', 'と', '思う', '」', 'と', '述べ', '、', '大量', '離党', 'に', 'は', '至ら', 'ない', 'と', 'の', '見通し', 'を', '示した', '。']] [16, 17] 0


In [18]:
t5_df = pd.DataFrame({
	"labels":ga_df['arg_surface'].str.cat(o_df['arg_surface'].str.cat(ni_df['arg_surface']))
})

In [19]:
t5_df

,labels
0,首相が
1,問題が影響を政権に
2,問題がことに
3,離党者が
4,問題が範囲に
...,...
108994,exogが責任を
108995,公職者が
108996,義務が悪習を
108997,うそが
